# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [25]:
# import libraries
from sqlalchemy import create_engine
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import precision_recall_fscore_support
import pickle

nltk.download(['punkt','stopwords','wordnet'])

[nltk_data] Downloading package punkt to /home/jyychang/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/jyychang/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/jyychang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [26]:
# load data from database
engine = create_engine('sqlite:///data/DisasterResponse.db')
df = pd.read_sql_table('Disasters', engine)
X = df['message']
y = df.iloc[:,4:] # not including 'id', 'message', 'original', 'genre'

In [32]:
X [500]

'Please, we are staying in a church. There are no wounded but we are in dire need of food, water, gas. Our house is completely leveled'

### 2. Write a tokenization function to process your text data

In [9]:
def tokenize(text):
    '''
    text [string] = sentence text
    Function to lowercase, tokenize, remove punctuations and stopwords, and lemmatize any string.
    lemmas [list of strings] = list of lemmas
    '''
    #normalize text
    text = re.sub(r'[^a-zA-Z0-9]',' ',text.lower())
    #tokenize messages
    words = word_tokenize(text)
    # remove stopwords
    tokens = [w for w in words if w not in stopwords.words("english")]
    # Lemmatization
    lemmas = [WordNetLemmatizer().lemmatize(t) for t in tokens]
    return lemmas

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [10]:
# Random Forest Classifier Pipeline
pipeline_rfc = Pipeline([
    ('vect', CountVectorizer(tokenizer=tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=43)

In [12]:
# Fit the Random Forest Classifier 
pipeline_rfc.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7ff7777ea290>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [16]:
def score_report(y_true, y_pred):
    '''
    y_true [dataframe] = dataframe with target labels as column headers and integer values 
    y_pred [array] = array of arrays of integers the same shape as y_true
    Function to compare predicted values to true values and calculate precision, recall, and f-score for each label.
    Function will also print the average precision, recall, and f-score across all labels.
    score_df = dataframe of labels, precision, recall, and f-score
    '''
    score_df = pd.DataFrame(columns=['Label', 'Precision', 'Recall', 'Fscore'])
    for i, col in enumerate(y_true.columns):
        precision, recall, f_score, support = precision_recall_fscore_support(y_true[col], y_pred[:, i], average='weighted')
        score_df.loc[i, 'Label'] = col
        score_df.loc[i, 'Precision'] = precision
        score_df.loc[i, 'Recall'] = recall
        score_df.loc[i, 'Fscore'] = f_score
    print('Average F Score: ', score_df['Fscore'].mean())
    print('Average Precision: ', score_df['Precision'].mean())
    print('Average Recall: ', score_df['Recall'].mean())
    return score_df

In [41]:
y_pred_rfc=pipeline_rfc.predict(X_test)
score_df_rfc = score_report(y_test, y_pred_rfc)

Average F Score:  0.9375561013929103
Average Precision:  0.9416491783157748
Average Recall:  0.949174380361442


/home/jyychang/DisasterResponsePipeline/pipeline_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [46]:
pipeline_rfc.get_params()

{'memory': None,
 'steps': [('vect',
   TfidfVectorizer(tokenizer=<function tokenize at 0x7f862a95e5f0>)),
  ('clf', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': TfidfVectorizer(tokenizer=<function tokenize at 0x7f862a95e5f0>),
 'clf': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.float64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__norm': 'l2',
 'vect__preprocessor': None,
 'vect__smooth_idf': True,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__sublinear_tf': False,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__use_idf': True,
 'vect__vocabulary': None,
 'clf__estimator__bootstrap': True,
 'clf__estim

In [44]:
score_df_rfc

,Label,Precision,Recall,Fscore
0,related,0.80848,0.818126,0.798489
1,request,0.894034,0.898383,0.887578
2,offer,0.990563,0.99527,0.992911
3,aid_related,0.78225,0.783338,0.782534
4,medical_help,0.910786,0.925236,0.898476
5,medical_products,0.947994,0.956363,0.94045
6,search_and_rescue,0.968193,0.974519,0.963843
7,security,0.968215,0.983979,0.976034
8,military,0.963579,0.972383,0.962118
9,child_alone,1,1,1


### 6. Improve your model
Use grid search to find better parameters. 

In [51]:
parameters_rfc = {'clf__estimator__min_samples_leaf': [1,10],
    'clf__estimator__max_features': ['auto','log2'],
              'clf__estimator__min_samples_split':[2, 10]}

cv_rfc = GridSearchCV(pipeline_rfc, param_grid=parameters_rfc)
cv_rfc.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x7f862a95e5f0>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'clf__estimator__max_features': ['auto', 'log2'],
                         'clf__estimator__min_samples_leaf': [1, 10],
                         'clf__estimator__min_samples_split': [2, 10]})

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [53]:
y_pred_cv_rfc = cv_rfc.predict(X_test)
score_cv_rfc = score_report(y_test, y_pred_cv_rfc)

Average F Score:  0.9375223994086507
Average Precision:  0.9415919329414808
Average Recall:  0.9491998101244365


/home/jyychang/DisasterResponsePipeline/pipeline_env/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [54]:
cv_rfc.best_estimator_

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x7f862a95e5f0>)),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [17]:
# Adaboost Classifier Pipeline
pipeline_abc = Pipeline([
    ('vect',TfidfVectorizer(tokenizer=tokenize)),
    ('clf',  MultiOutputClassifier(AdaBoostClassifier()))
    ])

In [18]:
# Train the Adaboost Classifier 
pipeline_abc.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x7ff7777ea290>)),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [19]:
# Test the pipeline
y_pred_abc=pipeline_abc.predict(X_test)
score_df_abc = score_report(y_test, y_pred_abc)

Average F Score:  0.9384573643905784
Average Precision:  0.9389071288091687
Average Recall:  0.9466822635879701


In [47]:
pipeline_abc.get_params()

{'memory': None,
 'steps': [('vect',
   TfidfVectorizer(tokenizer=<function tokenize at 0x7f862a95e5f0>)),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'vect': TfidfVectorizer(tokenizer=<function tokenize at 0x7f862a95e5f0>),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.float64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__norm': 'l2',
 'vect__preprocessor': None,
 'vect__smooth_idf': True,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__sublinear_tf': False,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__use_idf': True,
 'vect__vocabulary': None,
 'clf__estimator__algorithm': 'SAMME.R',
 'clf__estimato

In [45]:
score_df_abc

,Label,Precision,Recall,Fscore
0,related,0.743901,0.761215,0.683847
1,request,0.876994,0.885719,0.876071
2,offer,0.990557,0.994202,0.992376
3,aid_related,0.758162,0.759384,0.755068
4,medical_help,0.907974,0.923558,0.911997
5,medical_products,0.957861,0.963534,0.958325
6,search_and_rescue,0.966,0.973756,0.967912
7,security,0.973067,0.982148,0.976599
8,military,0.971388,0.974977,0.972645
9,child_alone,1,1,1


In [20]:
# Improve the pipeline
parameters_abc = {'vect__use_idf': (True, False),
        'clf__estimator__n_estimators': [40, 50, 60],
        'clf__estimator__learning_rate': [0.5, 1] }
cv_abc = GridSearchCV(pipeline_abc, param_grid = parameters_abc)
cv_abc.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(tokenizer=<function tokenize at 0x7ff7777ea290>)),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__learning_rate': [0.5, 1],
                         'clf__estimator__n_estimators': [40, 50, 60],
                         'vect__use_idf': (True, False)})

In [21]:
# Test Improvements
y_pred_cv_abc = cv_abc.predict(X_test)
score_cv_abc = score_report(y_test, y_pred_cv_abc)

Average F Score:  0.9362077857978802
Average Precision:  0.940036978988076
Average Recall:  0.9472968161936731


In [22]:
cv_abc.best_estimator_

Pipeline(steps=[('vect',
                 TfidfVectorizer(tokenizer=<function tokenize at 0x7ff7777ea290>,
                                 use_idf=False)),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(learning_rate=0.5,
                                                                    n_estimators=60)))])

### 9. Export your model as a pickle file

In [59]:
with open('models/abc_model.pkl', 'wb') as file:
    pickle.dump(pipeline_abc, file)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.